In [165]:
import numpy as np
import cv2
import os
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.legacy import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

## Extracting frames from the video clips

In [166]:
def extract_frames(video_path, output_folder, player_name, num_frames=20):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = total_frames // num_frames if total_frames > num_frames else 1

    video_basename = os.path.splitext(os.path.basename(video_path))[0]  # Get video file name without extension
    player_folder = os.path.join(output_folder, player_name)

    try:
        if not os.path.exists(player_folder):
            os.makedirs(player_folder)
    except OSError:
        print(f"Error: Creating directory {player_folder}")

    frame_ids = [int(interval * i) for i in range(num_frames)]
    frame_count = 0
    saved_frames = 0

    while saved_frames < num_frames:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_ids[saved_frames])
        success, frame = cap.read()
        if not success:
            print(f"Failed to read frame at index {frame_ids[saved_frames]} from {video_path}")
            saved_frames += 1
            continue

        frame_path = os.path.join(player_folder, f"{video_basename}_frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)
        saved_frames += 1
        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()


## Loading the extracted frames and preprocessing

In [167]:
def load_and_preprocess_data(root_folder):
    images = []
    labels = []
    valid_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".tiff")

    # Iterate through each player's folder to load images
    for player_name in os.listdir(root_folder):
        player_folder = os.path.join(root_folder, player_name)
        if os.path.isdir(player_folder):
            for img_file in os.listdir(player_folder):
                if img_file.lower().endswith(valid_extensions):
                    img_path = os.path.join(player_folder, img_file)
                    img = cv2.imread(img_path)
                    if img is None:
                        print(f"Warning: Could not read image {img_path} - it may be corrupt or in an unsupported format.")
                        continue

                    # img = cv2.resize(img, (64, 64))
                    img = cv2.resize(img, (128, 128))  # Resize for MobileNetV2
                    images.append(img)
                    labels.append(player_name)  # Label is the player's name

    images = np.array(images)
    labels = np.array(labels)

    # Encode labels to integers
    encoder = LabelEncoder()
    encoded_labels = encoder.fit_transform(labels)
    categorical_labels = to_categorical(encoded_labels)  # One-hot encoding

    num_images = len(images)
    print(f"Number of loaded and preprocessed frames: {num_images}")
    return images, categorical_labels, encoder, num_images

In [170]:
# def build_model(num_classes):
#     model = Sequential([
#         Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
#         MaxPooling2D(2, 2),
#         Conv2D(64, (3, 3), activation='relu'),
#         MaxPooling2D(2, 2),
#         Flatten(),
#         Dense(128, activation='relu'),
#         Dense(num_classes, activation='softmax')
#     ])
    
#     model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
#     return model



def build_model(num_classes):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False  # Freeze the layers of the base model
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [171]:
def main():
    root_folder = '/Users/nadunsenarathne/Downloads/Documents/IIT/4th Year/FYP/CricXpert/Datasets/India'
    output_frame_folder = '/Users/nadunsenarathne/Downloads/Documents/IIT/4th Year/FYP/CricXpert/Hybrid_Spatio_Temporal_Model_For_Gait_Analysis/extracted_frames'
    video_extensions = ('.mp4', '.avi', '.mov', '.mpeg', '.mpg', '.mkv')

    # for player_name in os.listdir(root_folder):
    #     player_path = os.path.join(root_folder, player_name, 'gait_data')
    #     if os.path.isdir(player_path):
    #         print(f"Processing videos for player: {player_name}")
    #         for video_file in os.listdir(player_path):
    #             if video_file.endswith(video_extensions):
    #                 video_path = os.path.join(player_path, video_file)
    #                 print(f"Extracting frames from: {video_path}")
    #                 extract_frames(video_path, output_frame_folder, player_name, 20)
    #             else:
    #                 print(f"Skipped non-video file: {video_file}")


    # Load and preprocess data
    X_train, y_train, encoder, num_images = load_and_preprocess_data(output_frame_folder)
 
    # Build and train model
    model = build_model(len(encoder.classes_))  # Ensure the model is built for the correct number of classes
    model.fit(X_train, y_train, epochs=10, batch_size=32)

    # Save the model and label encoder
    model.save('spatial_model.h5')
    np.save('label_encoder_classes.npy', encoder.classes_)


if __name__ == "__main__":
    main()

Number of loaded and preprocessed frames: 1749
Epoch 1/10
55/55 [==============================] - 5s 70ms/step - loss: 1.6650 - accuracy: 0.4260
Epoch 2/10
55/55 [==============================] - 4s 64ms/step - loss: 1.0030 - accuracy: 0.6810
Epoch 3/10
55/55 [==============================] - 4s 65ms/step - loss: 0.7464 - accuracy: 0.7759
Epoch 4/10
55/55 [==============================] - 4s 65ms/step - loss: 0.5230 - accuracy: 0.8553
Epoch 5/10
55/55 [==============================] - 4s 65ms/step - loss: 0.4222 - accuracy: 0.8902
Epoch 6/10
55/55 [==============================] - 4s 65ms/step - loss: 0.3282 - accuracy: 0.9291
Epoch 7/10
55/55 [==============================] - 4s 65ms/step - loss: 0.2560 - accuracy: 0.9485
Epoch 8/10
55/55 [==============================] - 4s 66ms/step - loss: 0.1983 - accuracy: 0.9708
Epoch 9/10
55/55 [==============================] - 4s 71ms/step - loss: 0.1767 - accuracy: 0.9703
Epoch 10/10
55/55 [==============================] - 4s 66ms/s

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Inference

In [174]:
import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

# Function to extract frames
def extract_frames_for_prediction(video_path, output_folder, num_frames=20):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = total_frames // num_frames if total_frames > num_frames else 1
    frame_ids = [int(interval * i) for i in range(num_frames)]
    frames = []

    try:
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
    except OSError:
        print(f"Error: Creating directory {output_folder}")

    frame_count = 0
    while frame_count < total_frames:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count in frame_ids:
            resized_frame = cv2.resize(frame, (128, 128))  # Resize frame as per training
            frames.append(resized_frame)
        frame_count += 1

    cap.release()
    return np.array(frames)


# Load the model
model = load_model('spatial_model.h5')

# Properly load the LabelEncoder
encoder = LabelEncoder()
encoder.classes_ = np.load('label_encoder_classes.npy', allow_pickle=True)

# Prediction function
def predict_player(video_path):
    frame_folder = 'temp_frames'
    frames = extract_frames_for_prediction(video_path, frame_folder)
    if len(frames) == 0:
        return "No frames to analyze."

    predictions = model.predict(frames)
    predicted_class = np.argmax(np.mean(predictions, axis=0))
    predicted_player = encoder.inverse_transform([predicted_class])[0]  # Translate label index back to player name

    return predicted_player

# Example usage
video_path = '/Users/nadunsenarathne/Downloads/Documents/IIT/4th Year/FYP/CricXpert/Datasets/untitled folder/2.mov'  # Path to your test video clip
result = predict_player(video_path)
print(f"The player in the video is: {result}")


1/1 [==============================] - 0s 396ms/step
The player in the video is: Virat_Kohli
